In [ ]:
import os
import sys

MODULE_PATH = '/content/drive/MyDrive/GitHub/DL_Study/Base'

sys.path.insert(0, MODULE_PATH)
sys.path

In [ ]:
# import
import numpy
from config import *
from optim import Adam
from models import LstmModelReg

# for time series split
!pip install scikit-learn==0.24.2

In [ ]:
# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [30]
    batch_size = [64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = LstmModelReg(time_size=24, hidden_size=64, feature_size=20)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

In [ ]:
import zipfile, requests, io
import pandas as pd
import numpy
import time
from datetime import datetime

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00580/hungary_chickenpox.zip'
r = requests.get(data_url)
files = zipfile.ZipFile(io.BytesIO(r.content))
df = pd.read_csv(files.open('hungary_chickenpox.csv'), sep=',')
df.drop('Date', axis=1, inplace=True)
df.head()

In [ ]:
from scipy.stats import skew, kurtosis
from statsmodels.tsa.stattools import adfuller

# jb = (n/6)*(skewness**2 + (kurtosis**2/4))

def data_statistics(df):
    df = df.dropna()
    data = df.values
    num = len(df)
    skewness_ = skew(data)
    kurtosis_ = kurtosis(data)
    jarque_bera_ = (num/6)*(skewness_**2 + (kurtosis_**2/4))
    result = adfuller(data)
    adf_ = result[0]
    print(f'skewness : {skewness_}')
    print(f'kurtosis : {kurtosis_}')
    print(f'jarque bera : {jarque_bera_}')
    print(f'ADF : {adf_}')

data_statistics(df['BUDAPEST'])

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 20
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

In [ ]:
n_splits = 3
test_size = (int)(len(new_df)*0.2)
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs, test_size=test_size).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []
i = 1

print('config : epochs, batch_size, learning_rate')

# neted cross validation
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = len(test_cv_X)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1,  n_inputs, n_features)
    val_X = val_X.reshape(-1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1,  n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        model.reset_state()
        predicted = model.predict(val_X)
        if GPU:
            predicted = np.asnumpy(predicted)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f' error(RMSE):{error}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)

    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    selected_model.reset_state()
    predicted = selected_model.predict(test_cv_X)
    if GPU:
        predicted = np.asnumpy(predicted)

    rmse = np.sqrt(MSE_metric(predicted, test_cv_y))
    mse = MSE_metric(predicted, test_cv_y)
    mae = MAE_metric(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse)
    best_mae.append(mae)   

    # model eval
    print(f'train-size:{train_X.shape[0]}, val-size:{val_X.shape[0]}, test-size:{test_cv_X.shape[0]}')
    print(f'best_model => error(rmse) : {rmse.item():.2f}, param:{configs[param]}')
    print()

In [ ]:
selected_model.reset_state()
predicted = selected_model.predict(test_cv_X)
if GPU:
    predicted = np.asnumpy(predicted)

def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)